In [1]:
import easylab_new2 as lab
import numpy as np
%matplotlib inline

In [2]:
x = lab.Var("x", lab.decimal_var_type(2))
y = lab.Var("y", lab.decimal_var_type(2))
z = lab.Var("z", lab.decimal_var_type(2), metadata={"source": "test"})

assert x.matches(x)
assert x.matches(...)
assert x.matches("*")
assert x.matches("x")
assert not x.matches(y)
assert not x.matches("y")

assert x.sub(1).matches(x.sub(1)) # Match by label comparison
assert x.sub(1).matches("x_1")
assert not x.sub(1).matches(x)
assert not x.sub(1).matches(x.sub(2))

assert x.matches("label: x")
assert x.matches("type: decimal(prec=2)")

assert z.matches("metadata: source: test")
assert not z.matches("metadata: source: notest")

In [3]:
record = lab.Record({x: 1, y: 2, z.sub(1): 3, z.sub(2): 4, z.sub(3): 5})
print(record)

{
    x = 1.00,
    y = 2.00,
    z_1 = 3.00,
    z_2 = 4.00,
    z_3 = 5.00
}


In [4]:
# Mutate record
record[x] = 2
print(record)

{
    x = 2.00,
    y = 2.00,
    z_1 = 3.00,
    z_2 = 4.00,
    z_3 = 5.00
}


In [5]:
assert not record.matches(x == 1)
assert record.matches(x == 2)
assert record.matches(z.sub(2) == 4)

assert record.matches(x > 1)
assert record.matches(x < 3)
assert not record.matches(x > 2)

# and
assert record.matches((x == 2) & (y == 2))
assert not record.matches((x == 2) & (y == 3))
# or
assert record.matches((x == 3) | (y == 2))
# not
assert record.matches(~(x == 3))


# shorthands
assert record.matches(x == y == 2)
assert not record.matches(x == y == 3)
assert record.matches(z.sub(1) < z.sub(2) < z.sub(3))
assert not record.matches(z.sub(1) < z.sub(2) > z.sub(3))

In [6]:
# Copy record
record2 = record.copy()
record2[y] = "42"
del record2[z.sub(2)]
print(record2)

assert record != record2

# Check that the original record is unchanged by comparing the records
print(record.compare(record2))

{ x = 2.00, y = 42.00, z_1 = 3.00, z_3 = 5.00 }
{ compare(y) = 2.00 ↔ 42.00, compare(z_2) = 4.00 ↔ }


In [7]:
print(record & record2)

{ x = 2.00, y = 2.00, z_1 = 3.00, z_3 = 5.00 }


In [8]:
# Computed
xy = lab.Computed("xy", [x, y], "x * y", float)

assert xy.type.value_type is float # Check type inference

assert xy(4, 3) == 12
assert xy.matches(xy)

# Record with computed
assert record[xy] == 4

In [9]:
data = lab.ListData([record, record2])
print(data)

[ {
    x = 2.00,
    y = 2.00,
    z_1 = 3.00,
    z_2 = 4.00,
    z_3 = 5.00
}, { x = 2.00, y = 42.00, z_1 = 3.00, z_3 = 5.00 } ]


In [10]:
record = data.where(y == 42).extract(lab.Record)

In [11]:
# Totzeit example

# infer all variables
totzeit = lab.load_data("totzeit.csv")
print(totzeit)
inferred_vars = totzeit.get_vars("metadata: source: inferred")
print(inferred_vars)

[
    { I [mA] = 0, R [Hz] = 2773 }
    { I [mA] = 0, R [Hz] = 4105 }
    { I [mA] = 0, R [Hz] = 5204 }
    { I [mA] = 0, R [Hz] = 6292 }
    { I [mA] = 0, R [Hz] = 7298 }
    ...13 more records (18 total)
]
[Var(label=Text({'ascii': 'I [mA]', 'unicode': 'I [mA]', 'latex': ' $I [mA]$ '}), type=<easylab_new2.data.var_type.VarType object at 0x000002461585F0D0>, metadata=Metadata(created_at=datetime.datetime(2022, 11, 13, 5, 22, 8, 621789), created_by='carlo', updated_at=None, updated_by=None, source='inferred')), Var(label=Text({'ascii': 'R [Hz]', 'unicode': 'R [Hz]', 'latex': ' $R [Hz]$ '}), type=<easylab_new2.data.var_type.VarType object at 0x000002461585F4F0>, metadata=Metadata(created_at=datetime.datetime(2022, 11, 13, 5, 22, 8, 622785), created_by='carlo', updated_at=None, updated_by=None, source='inferred'))]


In [12]:
# infer some variables
I = lab.Var("I", lab.PhysicalValue)
totzeit = lab.load_data("totzeit.csv", [I, "infer"])
print(totzeit)

[
    { I = 0.01, R [Hz] = 2773 }
    { I = 0.02, R [Hz] = 4105 }
    { I = 0.03, R [Hz] = 5204 }
    { I = 0.04, R [Hz] = 6292 }
    { I = 0.05, R [Hz] = 7298 }
    ...13 more records (18 total)
]


In [13]:
# infer no variables
R = lab.Var("R", lab.PhysicalValue)
totzeit = lab.load_data("totzeit.csv", [I, R])
print(totzeit)

[
    { I = 0.01, R = 2773.0 }
    { I = 0.02, R = 4105.0 }
    { I = 0.03, R = 5204.0 }
    { I = 0.04, R = 6292.0 }
    { I = 0.05, R = 7298.0 }
    ...13 more records (18 total)
]


In [14]:
U = lab.Computed("U", [I, R], "I * R", lab.PhysicalValue)
totzeit.add(U)
# totzeit.plot(U, R)

In [15]:
totzeit.inspect()

# Physical values

In [16]:


x = lab.value("1.4(1) m")
print(x)
t = lab.value("2.0(3)e-3s")
print(t)
v = lab.value("x / t", x=x, t=t)
print(v)
y = lab.value("cos(x)", x=x)
print(y)

(1.4 +/- 0.1)m
(0.0020 +/- 0.0004)s
700 +/- 120
0.170 +/- 0.099
